In [167]:
import json
import pandas as pd
from glob import glob
import os
import numpy as np

In [179]:
def get_raw_files():
    """ findes combines and splits to train dev test """
    all_jsons = glob('./**/*.json',recursive=True)
    for file_name in all_jsons:
        name = file_name.split(os.sep)[-1]
        if name == 'perspective_pool_v1.0.json':
            perspective = file_name
        elif name == 'evidence_pool_v1.0.json':
            evidence = file_name
        elif name == 'dataset_split_v1.0.json':
            split = file_name
        elif name == 'perspectrum_with_answers_v1.0.json':
            merger = file_name
            
    perspective = pd.read_json(perspective)
    perspective.columns= ['pId','perspective','source']
    evidence = pd.read_json(evidence)
    split = pd.read_json(split,typ='series').reset_index()
    split.columns = ['id','split']
    merger = pd.read_json(merger)
    return perspective,evidence,split,merger

def unwind_merger(merger_df):
    merger_df = merger_df.explode('perspectives')
    row_per_cluster = merger_df['perspectives'].apply(pd.Series)[['pids','stance_label_3']]
    row_per_cluster = row_per_cluster.explode('pids')
    row_per_cluster = row_per_cluster.drop_duplicates()
    row_per_pid_cid = pd.merge(merger_df,row_per_cluster,how='inner',left_index=True,right_index=True)
    row_per_pid_cid = row_per_pid_cid[['cId','text','pids','stance_label_3','split']]
    row_per_pid_cid = row_per_pid_cid.drop_duplicates()
    return row_per_pid_cid

def get_train_test_dev():
    """ main fubction """
    perspective,evidence,split,merger = get_raw_files()
    merger = pd.merge(merger,split,left_on='cId',right_on='id')
    merger = unwind_merger(merger)
    claim_and_pres = pd.merge(merger,perspective,left_on='pids',right_on='pId',how='inner')
    claim_pres_split = claim_and_pres[['text','perspective','stance_label_3','split']].drop_duplicates()
    train = claim_pres_split[claim_pres_split['split']=='train']
    test = claim_pres_split[claim_pres_split['split']=='test']
    dev = claim_pres_split[claim_pres_split['split']=='dev']
    return train,dev,test

In [180]:
train,dev,test = get_train_test_dev()

In [181]:
for i in (train,dev,test):
    print(len(i))

5096
1552
2425


In [183]:
dev.head()

,text,perspective,stance_label_3,split
0,Vaccination must be made compulsory,It is the state’s duty to protect its community,SUPPORT,dev
1,Vaccination must be made compulsory,The state must keep it's community safe.,SUPPORT,dev
2,Vaccination must be made compulsory,The safety of the community is the state's pri...,SUPPORT,dev
3,Vaccination must be made compulsory,Compulsory vaccination violates the individual...,UNDERMINE,dev
4,Vaccination must be made compulsory,Individuals have the right to refuse vaccinati...,UNDERMINE,dev


In [184]:
perspective,evidence,split,merger = get_raw_files()

In [187]:
split[split['id']==499]

,id,split
457,499,dev


In [186]:
merger.head()

,cId,text,source,perspectives,topics
0,499,Vaccination must be made compulsory,idebate,"[{'pids': [3695, 24076, 24077], 'stance_label_...",[health_and_medicine]
1,167,Male infant circumcision is tantamount to chil...,idebate,"[{'pids': [1242, 21983, 21984], 'stance_label_...","[culture, health_and_medicine, gender]"
2,943,School Day Should Be Extended,debatewise,"[{'pids': [7579], 'stance_label_3': 'SUPPORT',...",[]
3,944,punishment should fit the criminal,debatewise,"[{'pids': [7584, 26958, 26959], 'stance_label_...",[law]
4,621,A government has the obligation to lessen the ...,debatewise,"[{'pids': [5143], 'stance_label_3': 'SUPPORT',...","[politics, economy, society]"
